In [1]:
import duckdb

In [2]:
con = duckdb.connect(database=':memory:', read_only=False)

In [3]:
query = "SELECT * FROM 'customers.csv'"

In [4]:
result = con.execute(query)

In [5]:
df = result.fetchdf()
df.head()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

customer_id first_name last_name                        email  \
0         1001       John       Doe         john.doe@example.com   
1         1002      Emily     Smith    emily.smith@example.co.uk   
2         1003    Michael     Brown    michael.brown@example.com   
3         1004     Sophia    Wilson  sophia.wilson@example.co.th   
4         1005      David   Johnson    david.johnson@example.com   

           created_at created_date  status country  
0 2024-01-15 08:23:45   2024-01-15  active      US  
1 2024-02-10 14:12:30   2024-02-10  active      UK  
2 2024-03-05 19:45:12   2024-03-05  active      US  
3 2024-04-01 09:05:55   2024-04-01  active      TH  
4 2024-06-20 11:30:00   2024-06-20  active      US

In [6]:
query = "SELECT * FROM 'customers.csv'"

In [26]:
result = con.execute(query)

In [27]:
df = result.fetchdf()


In [9]:
df.head()

customer_id first_name last_name                        email  \
0         1001       John       Doe         john.doe@example.com   
1         1002      Emily     Smith    emily.smith@example.co.uk   
2         1003    Michael     Brown    michael.brown@example.com   
3         1004     Sophia    Wilson  sophia.wilson@example.co.th   
4         1005      David   Johnson    david.johnson@example.com   

           created_at created_date  status country  
0 2024-01-15 08:23:45   2024-01-15  active      US  
1 2024-02-10 14:12:30   2024-02-10  active      UK  
2 2024-03-05 19:45:12   2024-03-05  active      US  
3 2024-04-01 09:05:55   2024-04-01  active      TH  
4 2024-06-20 11:30:00   2024-06-20  active      US

In [10]:
query = "select * from read_json_auto('products.json')"

In [11]:

product_result = con.execute(query).fetchdf()

In [12]:
product_result.head()


id        name  price     category  in_stock
0   1      Laptop  35000  Electronics      True
1   2       Phone  18000  Electronics     False
2   3  Desk Chair   4500    Furniture      True

In [28]:
result = con.execute(query).fetchdf()

In [29]:
result.columns


Index(['customer_id', 'first_name', 'last_name', 'email', 'country',
       'created_date'],
      dtype='object')

In [14]:
query = """
SELECT
  customer_id,
  first_name,
  last_name,
  email,
  country,
  created_date
FROM 'customers.csv'
WHERE status = 'active'
  AND created_date >= '2024-01-01'
  AND country IN ('TH');
"""


In [15]:
thai_customers = con.execute(query).fetchdf()

In [30]:
customer_result2 = con.execute(query).fetchdf()

In [31]:
customer_result2 = con.execute(query).fetchdf()

customer_id first_name   last_name                           email country  \
0         1004     Sophia      Wilson     sophia.wilson@example.co.th      TH   
1         1008     Amanda         Lee        amanda.lee@example.co.th      TH   
2         1010      Nicha  Chaiyaphum  nicha.chaiyaphum@example.co.th      TH   

  created_date  
0   2024-04-01  
1   2024-09-02  
2   2025-02-14

In [32]:
query = """
SELECT
    PassengerId,
    Name,
    Age,
    Fare,
    CASE
        WHEN Survived = 1 THEN 'Survived'
        ELSE 'Not Survived'
    END as SurvivalStatus
FROM 'Titanic-Dataset.csv'
"""


In [33]:
transformed_data = con.execute(query).fetchdf()

In [34]:
transformed_data.head()

PassengerId                                               Name   Age  \
0            1                            Braund, Mr. Owen Harris  22.0   
1            2  Cumings, Mrs. John Bradley (Florence Briggs Th...  38.0   
2            3                             Heikkinen, Miss. Laina  26.0   
3            4       Futrelle, Mrs. Jacques Heath (Lily May Peel)  35.0   
4            5                           Allen, Mr. William Henry  35.0   

      Fare SurvivalStatus  
0   7.2500   Not Survived  
1  71.2833       Survived  
2   7.9250       Survived  
3  53.1000       Survived  
4   8.0500   Not Survived

In [35]:
query = """
SELECT
    c.customer_id,
    c.customer_name,
    c.segment,
    COUNT(DISTINCT o.order_id) as order_count,
    SUM(o.amount) as total_revenue,
    AVG(o.amount) as avg_order_value
FROM new_customers.csv c
LEFT JOIN orders.csv o
    ON c.customer_id = o.customer_id
    AND o.status != 'cancelled'
WHERE c.is_active = true
GROUP BY 1, 2, 3
"""

In [36]:
customer_metrics = con.execute(query).fetchdf()

In [37]:
customer_metrics.head()

customer_id customer_name     segment  order_count  total_revenue  \
0        C002    Jane Smith   Corporate            2          350.0   
1        C003     Acme Corp         SMB            1           80.0   
2        C010     Small Biz         SMB            1           45.0   
3        C001      John Doe    Consumer            2          195.5   
4        C005    Global LLC  Enterprise            2          800.0   

   avg_order_value  
0           175.00  
1            80.00  
2            45.00  
3            97.75  
4           400.00

In [38]:
query = "SELECT * FROM 'orders.csv'"

In [43]:
order_result = con.execute(query).fetchdf()
order_result.columns

Index(['order_id', 'customer_id', 'amount', 'order_date', 'status'], dtype='object')

In [44]:
query = "select * from 'new_customers.csv'"
new_customers_result = con.execute(query).fetchdf()
new_customers_result.columns


Index(['customer_id', 'customer_name', 'segment', 'is_active'], dtype='object')

In [48]:
query = """
WITH
-- ขั้นที่ 1: ดึงลูกค้าที่ active
active_customers AS (
    SELECT
        customer_id,
        customer_name,
        segment
    FROM new_customers.csv
    WHERE is_active = true
),

-- ขั้นที่ 2: คำนวณยอดสั่งซื้อ
order_metrics AS (
    SELECT
        customer_id,
        COUNT(*) as order_count,
        SUM(amount) as total_spent,
        AVG(amount) as avg_order_value
    FROM orders.csv
    WHERE order_date >= date_add(CURRENT_DATE, INTERVAL '-12 month')
    GROUP BY customer_id
),

-- ขั้นที่ 3: รวมข้อมูล
final AS (
    SELECT
        ac.*,
        COALESCE(om.order_count, 0) as orders_last_12m,
        COALESCE(om.total_spent, 0) as revenue_last_12m,
        om.avg_order_value
    FROM active_customers ac
    LEFT JOIN order_metrics om
        ON ac.customer_id = om.customer_id
)

SELECT * FROM final;
"""

result = con.execute(query).fetchdf()

In [51]:
result.head()

customer_id   customer_name     segment  orders_last_12m  revenue_last_12m  \
0        C007  Cancelled Cust    Consumer                1             100.0   
1        C009    Future Buyer  Enterprise                1            1000.0   
2        C010       Small Biz         SMB                1              45.0   
3        C001        John Doe    Consumer                0               0.0   
4        C002      Jane Smith   Corporate                0               0.0   

   avg_order_value  
0            100.0  
1           1000.0  
2             45.0  
3              NaN  
4              NaN

In [52]:
query = """
WITH enriched_orders AS (
    SELECT
        o.*,
        c.customer_name,
        c.segment
    FROM orders.csv o
    LEFT JOIN new_customers.csv c
        ON o.customer_id = c.customer_id
)
SELECT
    order_id,
    customer_id,
    customer_name,
    order_date,
    amount,

    -- 1. ลำดับของ order ของลูกค้าแต่ละคน
    ROW_NUMBER() OVER (
        PARTITION BY customer_id
        ORDER BY order_date
    ) as order_sequence,

    -- 2. อันดับยอดขายในเดือนนั้น
    RANK() OVER (
        PARTITION BY strftime('%Y-%m', order_date)
        ORDER BY amount DESC
    ) as monthly_sales_rank,

    -- 3. ยอดสะสมของลูกค้า
    SUM(amount) OVER (
        PARTITION BY customer_id
        ORDER BY order_date
    ) as running_total,

    -- 4. ค่าเฉลี่ยเคลื่อนที่ 4 orders
    AVG(amount) OVER (
        PARTITION BY customer_id
        ORDER BY order_date
        ROWS BETWEEN 3 PRECEDING AND CURRENT ROW
    ) as moving_avg_4_orders,

    -- 5. ยอด order ก่อนหน้า
    LAG(amount, 1, 0) OVER (
        PARTITION BY customer_id
        ORDER BY order_date
    ) as previous_order,

    -- 6. ยอด order ถัดไป
    LEAD(amount, 1, 0) OVER (
        PARTITION BY customer_id
        ORDER BY order_date
    ) as next_order,

    -- 7. ยอด order แรกของลูกค้า
    FIRST_VALUE(amount) OVER (
        PARTITION BY customer_id
        ORDER BY order_date
    ) as first_order_amount

FROM enriched_orders
ORDER BY customer_id, order_date;
"""


In [53]:
window_analysis = con.execute(query).fetchdf()

In [54]:
window_analysis.head()

order_id customer_id customer_name order_date  amount  order_sequence  \
0    O1001        C001      John Doe 2024-01-15   120.5               1   
1    O1002        C001      John Doe 2024-02-20    75.0               2   
2    O1003        C001      John Doe 2024-03-05    50.0               3   
3    O1004        C002    Jane Smith 2024-04-25   200.0               1   
4    O1005        C002    Jane Smith 2024-05-01   150.0               2   

   monthly_sales_rank  running_total  moving_avg_4_orders  previous_order  \
0                   1          120.5           120.500000             0.0   
1                   1          195.5            97.750000           120.5   
2                   1          245.5            81.833333            75.0   
3                   1          200.0           200.000000             0.0   
4                   1          350.0           175.000000           200.0   

   next_order  first_order_amount  
0        75.0               120.5  
1        50.0               120.5  
2         0.0               120.5  
3       150.0               200.0  
4         0.0               200.0